## Question 3: Natural Language Processing

In [11]:
corpuses = [
"nlp/01-introduction.csv",
"nlp/02-data-exploration.csv",
"nlp/03-decision-trees.csv",
"nlp/04-regression.csv",
"nlp/05-support-vector-machines.csv",
"nlp/06-neural-networks-1.csv",
"nlp/07-neural-networks-2.csv",
"nlp/08-evaluation.csv",
"nlp/09-clustering.csv",
"nlp/10-frequent-itemsets.csv",
]

In [ ]:
# Load the dataset
import pandas as pd


### a)

In [13]:
from collections import Counter
import plotly.express as px


### b)

In [ ]:
import nltk
from nltk.corpus import stopwords
import string



### c)

### d)

In [15]:
from nltk.util import ngrams
from nltk.probability import ConditionalFreqDist
import random


### e)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer
